In [ ]:
!pip install geopandas

In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

In [6]:
token = 'pk.eyJ1IjoianJzdmluY2Vuem8iLCJhIjoiY2wxMHc0ZWMyMDA1MDNqcnN4NDAxZHZxayJ9.XC2yFuBp7YjudgxK2Kjq7Q'
gdf = gpd.read_file('spainprovinces_def')
dataframe = pd.read_csv('acqxcatxtempo.csv',
                        sep = ';', encoding= 'unicode_escape')
geoprov = pd.read_csv('geoprov5.csv', sep = ';')
geoprov.rename(columns = {'prov_id' : 'customer_IDprovince'}, inplace = True)

In [3]:
def createbestseller():
  pivot = pd.pivot_table(dataframe, 
                         index = ['customer_IDprovince'], 
                         columns = ['category'], 
                         aggfunc = 'size', 
                         fill_value = 0)
  pivot_flat = pd.DataFrame(pivot.to_records())
  pivot_flat.set_index('customer_IDprovince', inplace = True)
  global bestseller_perprov
  bestseller_perprov = pd.concat([pivot_flat.index.to_series(),
                                pivot_flat.idxmax(axis=1),
                                pivot_flat.max(axis=1)],
                                 axis = 1).reset_index(drop = True)
  if 'Castellón' in pivot_flat:
    bestseller_perprov.iloc[47,0] = 'Castellón'
  bestseller_perprov.rename(columns = {0:'best_seller'}, inplace = True)
  bestseller_perprov = pd.merge(bestseller_perprov, 
                                geoprov, 
                                on ='customer_IDprovince', 
                                how ='inner')
  bestseller_perprov.rename(columns = {1:'count'}, inplace = True)

  global fig10
  fig10 = px.choropleth_mapbox(bestseller_perprov,
                               geojson=gdf,
                               featureidkey="properties.cod_prov",
                               locations="customer_IDprovince",
                               color="best_seller",
                               color_continuous_scale="matter",
                               zoom=3,
                               hover_name= 'province',
                               hover_data = ['count'],
                               center={"lat": 40.4999, "lon": -3.673},
                               labels={"best_seller": "Best Seller Category"})
  
  fig10.update_layout(mapbox_style="dark",
                      mapbox_accesstoken=token,
                      title_x = 0.5)

In [4]:
def createbestseller2():
  global fig1
  fig1 = px.scatter_mapbox(bestseller_perprov, lat = 'latprov', lon = 'lonprov',
                           size='count',
                           zoom=4, opacity = 0.9,
                           color = 'best_seller',
                           center = {"lat": 40.4167, "lon": -3.7167},
                           labels={"best_seller": "Best Seller Category"},
                           hover_name= 'province',
                           hover_data = ['count'])
  
  fig1.update_layout(mapbox_style="dark",
                     mapbox_accesstoken=token,
                     title_x = 0.5)

In [ ]:
while True:
  dataframe = pd.read_csv('acqxcatxtempo.csv',
                        sep = ';',
                        encoding= 'unicode_escape')
  
  print('Are you interested in all-time stats, by quarter or by year?')
  filter = str(input('Just write "AT", "Q", "Y":'))

  if filter == 'Q':
    year = str(input('Please specify the year you are interested in: (AA)'))
    quarter = str(input('Which quarter? (Q1):'))
    qy = quarter+year
    dataframe = dataframe[(dataframe['quadrimestre acquisto'] == qy)]
    
    bestseller_perprov = None
    fig10 = None
    createbestseller()
    createbestseller2()

    fig10.update_layout(title_text = 'Best selling products per province in {}'.format(qy))
    fig1.update_layout(title_text = 'Best selling products per province in {}'.format(qy))
    fig10.show()
    fig1.show()

  elif filter == 'Y':
    year = int(input('Please specify the year you are interested in: (AAAA)'))
    dataframe = dataframe[(dataframe['anno'] == year)]

    bestseller_perprov = None
    fig10 = None
    createbestseller()
    createbestseller2()

    fig10.update_layout(title_text = 'Best selling products per province in {}'.format(year))
    fig1.update_layout(title_text = 'Best selling products per province in {}'.format(year))
    fig10.show()
    fig1.show()

  else:
    bestseller_perprov = None
    fig10 = None
    fig1 = None
    createbestseller()
    createbestseller2()

    fig10.update_layout(title_text = 'All time best selling products per province')
    fig1.update_layout(title_text = 'All time best selling products per province')
    fig10.show()
    fig1.show()